In [1]:
## Import Dependencies and libraries

import numpy as np
import glob
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import datetime, calendar
from matplotlib.dates import DateFormatter as dtfmt
from alive_progress import alive_bar

# Reading septentrio file and concatenate

root = tk.Tk()
root.withdraw()
#chose year folder with day concatinated files
file_path = filedialog.askdirectory()

path = file_path+'/*.ismr'
files = glob.glob(path)
files

len(files)

355

In [2]:
# ISMR Data Processing

import pandas as pd
import datetime, calendar

#data= pd.read_csv("concated_{Folder_name}.ismr".format(Folder_name=file_path[-5:]),header=None) #import concated file of the day

#data = pd.read_csv('concated_19044.ismr',header=None)

##some functions
def PRN_X(x):
    sv = 'M' + str(x)
    if 0 < x < 38:
        sv = 'G' + str(x)
    elif 37 < x < 62:
        sv = 'R' + str(x)
    elif 70 < x < 107:
        if len(str(x)) > 2:
            x = str(x)[-2:]
        sv = 'E' + str(x)
    elif 119 < x < 139:
        if len(str(x)) > 2:
            x = str(x)[-2:]
        sv = 'S' + str(x)
    elif 140 < x < 177:
        if len(str(x)) > 2:
            x = str(x)[-2:]
        sv = 'C' + str(x)
    elif 181 < x < 187:
        if len(str(x)) > 2:
            x = str(x)[-2:]
        sv = 'J' + str(x)
    return sv


###==============================================###
###
def WN_TOWtoTIME(WN, TOW):
    """"
    Convert gpsweek and gpseconds to time
    """
    import datetime, calendar
    WN = float(WN)
    TOW = float(TOW)
    # print(gpsweek,gpsseconds)
    leapseconds = 0   ####check 
    datetimeformat = "%Y-%m-%d %H:%M:%S"
    epoch = datetime.datetime.strptime("1980-01-06 00:00:00", datetimeformat)
    elapsed = datetime.timedelta(days=(WN * 7), seconds=(TOW))
    # return datetime.datetime.strftime(epoch + elapsed,datetimeformat)
    return elapsed + epoch

In [9]:
with alive_bar(len(files),force_tty=True) as bar:
    for day_file in files[:4]:
        year=int(file_path[-4:])
        DAY_OF_YEAR = int(day_file[-8:-5])
        d=datetime.date(year, 1, 1) + datetime.timedelta(DAY_OF_YEAR - 1)
        data=pd.read_csv(day_file,header=None)
        data.columns=['WN','TOW','SVID', 'Value', 'Azimuth', 'Elevation', 'Sig1', 'Total_S4_Sig1',
               'Correction_total_S4_Sig1', 'Phi01_Sig1_1', 'Phi03_Sig1_3',
               'Phi10_Sig1_10', 'Phi30_Sig1_30', 'Phi60_Sig1_60',
               'AvgCCD_Sig1_average_code-carrier_divergence',
               'SigmaCCD_Sig1_standard_deviation_code-carrier_divergence',
               'TEC_TOW-45s', 'dTEC_TOW-60s_TOW-45s', 'TEC_TOW-30s',
               'dTEC_TOW-45s_TOW-30s', 'TEC_TOW-15s', 'dTEC_TOW-30s_TOW-15s',
               'TEC_TOW', 'dTEC_TOW-15s_TOW', 'Sig1_lock_time',
               'sbf2ismr_version_number', 'Lock_time_second_frequency_TEC',
               'Averaged_C/N0_second_frequency_TEC_computation', 'SI_Index_Sig1',
               'SI_Index_Sig1_numerator', 'p_Sig1_spectral_slope', 'Average_Sig2_C/N0',
               'Total_S4_Sig2', 'Correction_total_S4_Sig2', 'Phi01_Sig2_1',
               'Phi03_Sig2_3', 'Phi10_Sig2_10', 'Phi30_Sig2_30', 'Phi60_Sig2_60',
               'AvgCCD_Sig2_average_code-carrier_divergence', 'SigmaCCD_Sig2_standard',
               'Sig2_lock', 'SI_Index_Sig2', 'SI_Index_Sig2_numerator', 'p_Sig2_phase',
               'Average_Sig3_C/N0_last_minute', 'Total_S4_Sig3',
               'Correction_total_S4_Sig3', 'Phi01_Sig3_1_phase', 'Phi03_Sig3_3_phase',
               'Phi10_Sig3_10_phase', 'Phi30_Sig3_30_phase', 'Phi60_Sig3_60_phase',
               'AvgCCD_Sig3_average_code-carrier_divergence',
               'SigmaCCD_Sig3_standard_deviation_code-carrier_divergence',
               'Sig3_lock_time', 'SI_Index_Sig3', 'SI_Index_Sig3_numerator',
               'p_Sig3_phase', 'T_Sig1_phase', 'T_Sig2_phase', 'T_Sig3_phase']

        data['sv'] = data.SVID.apply(PRN_X)
        data['Time_UTC']=np.vectorize(WN_TOWtoTIME)(data['WN'],data['TOW'])
        data['Time_IST']=data.Time_UTC+datetime.timedelta(hours=5.5)
        # Time
        data['JT'] = pd.DatetimeIndex(data['Time_UTC']).floor('d').to_julian_date()


        #elevation cut-off
        data=data[data['Elevation'].astype(float) >=25]

        # Categorizing
        data_gps = data.loc[data['SVID'].between(1,37,inclusive ="both")]

        data_glonass = data.loc[data['SVID'].between(38,61,inclusive ="both")]

        data_gallileo = data.loc[data['SVID'].between(71,106,inclusive ="both")]


        data_SBAS = data.loc[data['SVID'].between(120,138,inclusive ="both")]


        data_compass = data.loc[data['SVID'].between(140,176,inclusive ="both")]


        data_Qzss = data.loc[data['SVID'].between(181,187,inclusive ="both")]



        # Scintillation Index

        #1
        '''max_S4_of_day=max(data_gps.Total_S4_Sig1)
        maxima=data_gps[data_gps.Total_S4_Sig1==max_S4_of_day]
        maxima

        svid=eval(maxima['SVID'].to_string(index=False))
        svid

        fig_size = [14, 8]
        plt.rcParams['figure.figsize'] = fig_size
        top_scinti=data_gps[data_gps.SVID==svid]
        plt.plot(top_scinti.Time_UTC,top_scinti.Total_S4_Sig1.astype(float))
        #plt.xlim(G31.Time_UTC(2019,2,13,2,0,0),G31.Time_UTC(2019,2,13,4,0,0)) #to set xlim (change acoordingly)
        plt.xticks(rotation=90)'''
        #2
        '''fig_size = [21, 17]
        plt.rcParams['figure.figsize'] = fig_size
        pd.pivot_table(data_gps.reset_index(),
                       index='Time_UTC', columns='SVID', values='Total_S4_Sig1'
                      ).plot(subplots=True,layout=(11,3),ylim=(0.0,1),title='Date: '+str(d))'''

        #3
        fig_size = [21, 12]

        ###########################################
        plt.rcParams['figure.figsize'] = fig_size
        plt.rcParams['ytick.minor.visible'] =True
        plt.rcParams['xtick.minor.visible'] = True
        plt.rcParams['xtick.top'] = True
        plt.rcParams['ytick.right'] = True
        plt.rcParams['font.size'] = '20'
        plt.rcParams['legend.fontsize'] = '13'
        plt.rcParams['legend.borderaxespad'] = '1.9'
        #plt.rcParams['legend.numpoints'] = '1'
        plt.rcParams['figure.titlesize'] = 'medium'
        plt.rcParams['figure.titlesize'] = 'medium'
        plt.rcParams['xtick.major.size'] = '10'
        plt.rcParams['xtick.minor.size'] = '6'
        plt.rcParams['xtick.major.width'] = '2'
        plt.rcParams['xtick.minor.width'] = '1'
        plt.rcParams['ytick.major.size'] = '10'
        plt.rcParams['ytick.minor.size'] = '6'
        plt.rcParams['ytick.major.width'] = '2'
        plt.rcParams['ytick.minor.width'] = '1'
        plt.rcParams['xtick.direction'] = 'in'
        plt.rcParams['ytick.direction'] = 'in'
        plt.rcParams['axes.labelpad'] = '10.0'
        plt.rcParams['lines.dashed_pattern']=3.0, 1.4
        #plt.rcParams['axes.formatter.limits']=-10,10
        plt.rcParams['lines.dotted_pattern']= 1.0, 0.7

        plt.rcParams['xtick.labelsize'] = '16'
        plt.rcParams['ytick.labelsize'] = '16'
        plt.rcParams['axes.labelsize'] = '16'
        plt.rcParams['axes.labelsize'] = '16'
        plt.rcParams['axes.labelweight'] = 'bold'

        plt.rcParams['xtick.major.pad']='10'
        plt.rcParams['xtick.minor.pad']='10'
        plt.rcParams['hatch.color'] = 'black'
        plt.rc('axes', linewidth=2)
        

        #############################################
        #####Trial Lines###

        ###################
        pvt_table=pd.pivot_table(data_gps.reset_index(),
                       index='Time_UTC', columns='SVID', values='Total_S4_Sig1')

        fig,axes=plt.subplots()
        axes.plot(pvt_table,alpha=0.5)
        axes.legend(pvt_table.columns,loc=6, bbox_to_anchor=(1, 0.5))
        axes.margins(x=0)
        plt.ylim(0.0,0.6)
        axes.set_xlim(data_gps.Time_UTC.min(),data_gps.Time_UTC.max())
        axes.xaxis.set_major_formatter(dtfmt('%H:%M\n%d-%b'))
        axes.set_xlabel("Time(UT)")
        axes.set_ylabel(r"$S_4$")
    
        ax2=axes.twiny() 
        ax2.set_xlim(data_gps.Time_IST.min(),data_gps.Time_IST.max())
        ax2.margins(x=0)
        ax2.xaxis.set_major_formatter(dtfmt('%d-%b\n%H:%M'))
        ax2.set_xlabel("Time (LT)")

        plt.savefig(f'Scintillation/{year}/{str(d)}.png', facecolor='white',dpi=500, bbox_inches='tight')
        plt.close()

        #gif
        #image = imageio.v2.imread(f'Scintillation_Plots2/{year}/{str(d)}.png')
        #frames.append(image)
        bar()

|▌⚠︎                                      | (!) 4/355 [1%] in 18.5s (0.19/s)                                             


In [4]:
data

,WN,TOW,SVID,Value,Azimuth,Elevation,Sig1,Total_S4_Sig1,Correction_total_S4_Sig1,Phi01_Sig1_1,...,SI_Index_Sig3,SI_Index_Sig3_numerator,p_Sig3_phase,T_Sig1_phase,T_Sig2_phase,T_Sig3_phase,sv,Time_UTC,Time_IST,JT
1,2199,172860,10,628,119.0,31.0,44.3,0.083,0.061,0.035,...,0.015,1.526,1.56,0.000167,0.000097,0.000060,G10,2022-03-01 00:01:00,2022-03-01 05:31:00,2459639.5
3,2199,172860,21,628,264.0,44.0,46.4,0.073,0.048,0.020,...,nan,nan,nan,0.000045,nan,nan,G21,2022-03-01 00:01:00,2022-03-01 05:31:00,2459639.5
4,2199,172860,22,628,21.0,44.0,45.9,0.057,0.051,0.020,...,nan,nan,nan,0.000053,nan,nan,G22,2022-03-01 00:01:00,2022-03-01 05:31:00,2459639.5
6,2199,172860,26,628,151.0,34.0,42.2,0.106,0.077,0.031,...,nan,nan,nan,0.000253,nan,nan,G26,2022-03-01 00:01:00,2022-03-01 05:31:00,2459639.5
7,2199,172860,31,628,27.0,73.0,48.2,0.062,0.039,0.024,...,nan,nan,nan,0.000069,nan,nan,G31,2022-03-01 00:01:00,2022-03-01 05:31:00,2459639.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31049,2199,259200,126,628,209.0,60.0,44.2,0.065,0.061,0.035,...,nan,nan,nan,0.000121,0.000148,nan,S26,2022-03-02 00:00:00,2022-03-02 05:30:00,2459640.5
31050,2199,259200,127,628,225.0,55.0,35.3,0.169,0.173,0.088,...,nan,nan,nan,0.000592,0.000141,nan,S27,2022-03-02 00:00:00,2022-03-02 05:30:00,2459640.5
31051,2199,259200,128,628,162.0,62.0,48.1,0.042,0.039,0.164,...,nan,nan,nan,0.000726,0.000719,nan,S28,2022-03-02 00:00:00,2022-03-02 05:30:00,2459640.5
31053,2199,259200,132,628,140.0,57.0,44.3,0.072,0.061,0.053,...,nan,nan,nan,0.000217,0.000107,nan,S32,2022-03-02 00:00:00,2022-03-02 05:30:00,2459640.5
